In this example we will see how to perform topic extraction using MiniSom. The goal is to extract the main topics (represented as a set of words) that occur in a collection of documents.

In [1]:
import sys
sys.path.insert(0, '/home/zephyr/Paper2Agent-main/Minisom_Agent/repo/minisom/')

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 300
plt.rcParams["savefig.dpi"] = 300

from minisom import MiniSom
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

The colloction of documents that we will work with is the famous `20newsgroups` dataset. It contains more than 10000 newsgroups posts. We will download the dataset using sklearn and will transform the textual documents into a matrix `D` where each row represents a post using <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer">TF-IDF representation</a>:

In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
documents = dataset.data

no_features = 1000

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=no_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
D = tfidf.todense().tolist()

Now we have to train a SOM that clusters the documents, the total number of neurons in the SOM will be also the number of topics to extract:

In [3]:
n_neurons = 2
m_neurons = 4
som = MiniSom(n_neurons, m_neurons, no_features)
som.random_weights_init(D)
som.train(D, 5000, random_order=False, verbose=True)

 [ 2027 / 5000 ]  41% - 0:00:00 left 

 [ 3669 / 5000 ]  73% - 0:00:00 left 

 [ 5000 / 5000 ] 100% - 0:00:00 left 


 quantization error: 0.9884376329597593


We will consider as topic the list of first `top_keywords` associated with the biggest weights of each neuron. With the following for loop we will inspect all the weights and recover the words associated with the weights using the feature names saved by the TfidfVectorizer:

In [4]:
top_keywords = 10

weights = som.get_weights()
cnt = 1
for i in range(n_neurons):
    for j in range(m_neurons):
        keywords_idx = np.argsort(weights[i,j,:])[-top_keywords:]
        keywords = ' '.join([tfidf_feature_names[k] for k in keywords_idx])
        print('Topic', cnt, ':', keywords)
        cnt += 1

Topic 1 : pc times point remember year way time better max old
Topic 2 : said idea month goal guns group police gun government hell
Topic 3 : mail israel position interested exists question gov course included new
Topic 4 : know chicago file widget window files manager running dos windows
Topic 5 : players christians thought need really christ hockey action don god
Topic 6 : used cause turkish gas just israeli basic apparently results people
Topic 7 : couple like modem day mention got drive steve questions ve
Topic 8 : says heard knows sure wanted talking team kill knew men
